In [ ]:
import pandas as pd
import openpyxl
import os
import re
import openai
from tqdm import tqdm
from openai import AzureOpenAI
from dotenv import load_dotenv
import json
from tqdm.contrib.concurrent import thread_map
from langchain_openai import AzureChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
from tqdm.contrib.concurrent import thread_map
from sklearn.metrics import classification_report

pd.set_option('display.max_colwidth', 1)

In [ ]:
import os

os.environ["AZURE_OPENAI_ENDPOINT"] = ""
os.environ["AZURE_OPENAI_API_KEY"] = ""
os.environ["OPENAI_API_VERSION"] = ""

In [ ]:
load_dotenv('openai.env')

azure_endpoint = os.getenv('AZURE_OPENAI_ENDPOINT')  
azure_api_key = os.getenv('AZURE_OPENAI_API_KEY')  
openai_api_version = os.getenv('OPENAI_API_VERSION')  

In [ ]:
llm = AzureChatOpenAI(azure_deployment="artigo_stil",temperature=0)

In [ ]:
def run_gpt_ajustado(text):
    PROMPT_CLASSIFICATION = "Você está analisando uma postagem de uma rede social. Caso o texto contenha discurso de ódio, ofensas, agressões, xingamentos, palavrões ou qualquer forma de toxicidade, responda apenas com 'sim'. Se for adequado e sem toxicidade, responda 'não'." + text
    try:
        result = llm.invoke(PROMPT_CLASSIFICATION.format(query=text))
        return result.content
    except Exception as e:
        if "The response was filtered due to the prompt triggering Azure OpenAI's content management policy" in str(e):
            return "erro_content_moderation"
        else:
            raise e


In [ ]:
run_gpt_ajustado('oi')

In [ ]:
data = pd.read_csv('test_toldbr.csv')

In [ ]:
data

In [ ]:
tqdm.pandas(desc="Classificando textos")
data['gpt4omini_ajustado'] = data['clean_text'].progress_apply(run_gpt_ajustado)

In [ ]:
data.to_csv('test_told_result_gptasjutado_1.csv', index=False)

In [ ]:
data = data_gpt

In [ ]:
data_gpt.Toxic.loc[data_gpt.Toxic==1.0]='sim'
data_gpt.Toxic.loc[data_gpt.Toxic==0.0]='não'

In [ ]:
data_gpt

In [ ]:
print(classification_report(data_gpt['Toxic'], data_gpt['gpt4omini_ajustado']))